##### _Asset Price Data Import and Generator of Mean Return Vector & Covariance Matrix for Portfolio Optimisation_

In [2]:
import pandas as pd
import numpy as np
import sqlite3

conn = sqlite3.connect('continuous_futures.db')

##### _SQL Query to import specific set of assets & date range from database:_

In [3]:
assets = pd.read_sql("SELECT date, quandl_code, settle FROM \
Continuous_Futures WHERE method = 'FW' AND depth = 1 AND exchange = 'CME' \
AND date BETWEEN '2019-01-01' AND '2019-03-01'", conn)

In [ ]:
assets.set_index('date', inplace = True)

assets_p = assets.pivot(columns="quandl_code", values="settle")

assets_p.sort_values(by = 'date', ascending = False, inplace = True)

In [ ]:
# Check whether there are a significant number of missing price values for certain assets within the chosen date range
assets_p.isna().sum()

In [63]:
asset_list = assets_p.columns[1:].to_list()

daily_returns = pd.DataFrame()

In [64]:
for i in asset_list:
    daily_returns[f'{i}_long'] = ((assets_p[f'{i}'] - assets_p[f'{i}'].shift(-1)) / assets_p[f'{i}'].shift(-1))
    daily_returns[f'{i}_short'] = ((assets_p[f'{i}'].shift(-1) - assets_p[f'{i}']) / assets_p[f'{i}'].shift(-1))

daily_returns.dropna(inplace = True)

In [65]:
# Create vector of mean daily return for each asset, long & short

mean_return = []

for i in asset_list:
    long_return = daily_returns[f'{i}_long'].mean()
    short_return = daily_returns[f'{i}_short'].mean()
    mean_return.append(long_return)
    mean_return.append(short_return)

return_array = np.array(mean_return)

# Create covariance matrix for asset daily return data

covariance_matrix = daily_returns.cov().to_numpy()

In [66]:
np.savetxt("c_matrix.csv", covariance_matrix, delimiter=",")
np.savetxt("m_return.csv", return_array, delimiter=",")